In [1]:
from collections import defaultdict
import numpy as np
import os
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
from plotly.subplots import make_subplots
import statsmodels.formula.api as smf

from data import load_pcd_df
from plotting import save_plot

pio.templates.default = "plotly_white"

In [2]:
results_dir = 'results/2025-02-13/'
os.makedirs(results_dir, exist_ok=True)

In [3]:
save = True
benchmarks = ['MMLU', 'GPQA Diamond', 'MATH 5', 'MATH-500', 'HumanEval']
benchmark_is_mqa = {'MMLU': True, 'GPQA Diamond': True, 'MATH 5': False, 'MATH-500': False, 'HumanEval': False, 'LMSys Chatbot Arena ELO': False}
min_num_data_points_for_regression = 4

# Using Artificial Analysis data

## Load Artificial Analysis dataset

In [4]:
aa_df = pd.read_csv('data/aa_data_with_math5.csv')
aa_df

,Model Name,Release Date,USD per 1M Tokens,Tokens per Second,Prefill Latency (s),Prompt Length (tk),MMLU,GPQA Diamond,MATH-500,HumanEval,LMSys Chatbot Arena ELO,MATH 5
0,Claude-3-Haiku,2024-03,0.5000,122.7,0.467,1000.0,71,33,39.0,77.0,1179.0,13.0
1,Claude-3-Opus,2024-03,30.0000,26.5,1.984,1000.0,84,50,64.0,83.0,1248.0,34.0
2,Claude-3-Sonnet,2024-03,6.0000,61.8,0.789,1000.0,77,37,41.0,71.0,1201.0,16.0
3,Claude-3.5-Haiku,2024-06,1.6000,64.2,0.768,1000.0,81,37,67.0,87.0,NaN,NaN
4,Claude-3.5-Sonnet-2024-06,2024-06,6.0000,55.9,0.906,1000.0,88,56,71.0,90.0,1268.0,46.0
5,Claude-3.5-Sonnet-2024-10,2024-10,6.0000,55.2,0.907,1000.0,89,58,76.0,96.0,1282.0,53.0
6,Gemini 2.0 Flash,2025-02,0.1750,NaN,NaN,NaN,88,62,93.0,90.0,NaN,82.0
7,Gemini-1.5-Flash-2024-05,2024-05,0.1300,298.4,0.307,1000.0,79,39,55.0,NaN,1227.0,23.0
8,Gemini-1.5-Flash-2024-09,2024-09,0.1300,190.5,0.348,1000.0,75,45,83.0,83.0,1271.0,58.0
9,Gemini-1.5-Flash-8B,2024-10,0.0700,285.2,0.335,1000.0,75,30,70.0,12.0,1211.0,NaN


In [5]:
# 'Release Date' is a string with the format 'YYYY-MM'
aa_df['Release Date'] = pd.to_datetime(aa_df['Release Date'].str.strip(), format='%Y-%m')

## Load Epoch AI price dataset

In [6]:
api_price_df = pd.read_csv('data/epoch_ai_price_data_not_in_aa_with_benchmarks.csv')
api_price_df['Release Date'] = pd.to_datetime(api_price_df['Release Date'].str.strip(), format='%Y-%m-%d')
api_price_df

,Model Name,Release Date,USD per 1M Tokens,Tokens per Second,Prefill Latency (s),Prompt Length (tk),MMLU,GPQA Diamond,MATH 5,MATH-500,HumanEval,LMSys Chatbot Arena ELO
0,Claude 2,2024-08-12,12.0000,NaN,NaN,NaN,78.5,35.0,10.0,NaN,NaN,NaN
1,Claude 2.1,2024-08-12,12.0000,NaN,NaN,NaN,NaN,36.0,11.0,NaN,NaN,NaN
2,Claude Instant,2024-08-12,1.2000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Cohere Command,2024-08-13,1.6250,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Cohere Command Light,2024-08-13,0.3750,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Command R,2024-08-13,0.7500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Command R+,2024-08-13,6.0000,NaN,NaN,NaN,75.7,NaN,NaN,NaN,NaN,NaN
7,Command R+,2024-09-13,4.3750,NaN,NaN,NaN,75.7,NaN,NaN,NaN,NaN,NaN
8,DeepSeek-Coder-V2 236B,2024-09-11,0.1750,NaN,NaN,NaN,79.2,NaN,NaN,NaN,NaN,NaN
9,DeepSeek-R1,2025-01-20,0.9600,NaN,NaN,NaN,NaN,71.7,93.1,NaN,NaN,NaN


In [7]:
# Merge the two datasets
aa_df = pd.concat([aa_df, api_price_df])
aa_df.sort_values(by='Release Date', inplace=True)
# Reset the index
aa_df.reset_index(drop=True, inplace=True)
aa_df

,Model Name,Release Date,USD per 1M Tokens,Tokens per Second,Prefill Latency (s),Prompt Length (tk),MMLU,GPQA Diamond,MATH-500,HumanEval,LMSys Chatbot Arena ELO,MATH 5
0,GPT-3 175B (davinci),2021-11-20,60.000,NaN,NaN,NaN,43.9,NaN,NaN,NaN,NaN,NaN
1,GPT-3 175B (davinci),2022-08-31,60.000,NaN,NaN,NaN,43.9,NaN,NaN,NaN,NaN,NaN
2,GPT-3 175B (davinci),2022-09-01,20.000,NaN,NaN,NaN,43.9,NaN,NaN,NaN,NaN,NaN
3,GPT-3.5,2022-11-30,20.000,NaN,NaN,NaN,64.8,NaN,NaN,NaN,NaN,NaN
4,GPT-4,2023-03-01,37.500,23.6,0.724,1000.0,86.0,33.0,21.0,67.0,1186.0,23.0
...,...,...,...,...,...,...,...,...,...,...,...,...
66,Mistral Small 3,2025-01-01,0.475,NaN,NaN,NaN,82.0,46.0,74.0,85.0,NaN,45.0
67,DeepSeek-R1,2025-01-20,0.960,NaN,NaN,NaN,NaN,71.7,NaN,NaN,NaN,93.1
68,o3-mini,2025-01-31,1.925,NaN,NaN,NaN,NaN,74.3,NaN,NaN,NaN,95.2
69,o1-mini,2025-01-31,1.925,NaN,NaN,NaN,NaN,59.5,NaN,NaN,NaN,84.3


In [8]:
# Rename all instances of 'GPT-3 175B (davinci)' to 'GPT-3'
aa_df.loc[aa_df['Model Name'] == 'GPT-3 175B (davinci)', 'Model Name'] = 'GPT-3'
aa_df


,Model Name,Release Date,USD per 1M Tokens,Tokens per Second,Prefill Latency (s),Prompt Length (tk),MMLU,GPQA Diamond,MATH-500,HumanEval,LMSys Chatbot Arena ELO,MATH 5
0,GPT-3,2021-11-20,60.000,NaN,NaN,NaN,43.9,NaN,NaN,NaN,NaN,NaN
1,GPT-3,2022-08-31,60.000,NaN,NaN,NaN,43.9,NaN,NaN,NaN,NaN,NaN
2,GPT-3,2022-09-01,20.000,NaN,NaN,NaN,43.9,NaN,NaN,NaN,NaN,NaN
3,GPT-3.5,2022-11-30,20.000,NaN,NaN,NaN,64.8,NaN,NaN,NaN,NaN,NaN
4,GPT-4,2023-03-01,37.500,23.6,0.724,1000.0,86.0,33.0,21.0,67.0,1186.0,23.0
...,...,...,...,...,...,...,...,...,...,...,...,...
66,Mistral Small 3,2025-01-01,0.475,NaN,NaN,NaN,82.0,46.0,74.0,85.0,NaN,45.0
67,DeepSeek-R1,2025-01-20,0.960,NaN,NaN,NaN,NaN,71.7,NaN,NaN,NaN,93.1
68,o3-mini,2025-01-31,1.925,NaN,NaN,NaN,NaN,74.3,NaN,NaN,NaN,95.2
69,o1-mini,2025-01-31,1.925,NaN,NaN,NaN,NaN,59.5,NaN,NaN,NaN,84.3


## Explore the data

In [9]:
# Plot 'MMLU' vs. 'USD per 1M Tokens'
fig = px.scatter(aa_df, x='MMLU', y='USD per 1M Tokens', hover_data=['Model Name'], title='MMLU vs. USD per 1M Tokens')
fig.update_layout(yaxis_type='log')
fig.show()


In [10]:
aa_df['MMLU price-performance'] = aa_df['MMLU'] / aa_df['USD per 1M Tokens']
# Plot 'MMLU price-performance' vs. 'Date'
fig = px.scatter(aa_df, x='Release Date', y='MMLU price-performance', title='MMLU price-performance over time', hover_data=['Model Name'])
fig.update_layout(yaxis_type='log')
fig.show()


In [11]:
aa_df['Model Name and Date'] = aa_df['Model Name'] + ' (' + aa_df['Release Date'].dt.strftime('%Y-%m') + ')'
aa_df['Model Name and Date']

0                GPT-3 (2021-11)
1                GPT-3 (2022-08)
2                GPT-3 (2022-09)
3              GPT-3.5 (2022-11)
4                GPT-4 (2023-03)
                 ...            
66     Mistral Small 3 (2025-01)
67         DeepSeek-R1 (2025-01)
68             o3-mini (2025-01)
69             o1-mini (2025-01)
70    Gemini 2.0 Flash (2025-02)
Name: Model Name and Date, Length: 71, dtype: object

In [12]:
# Construct a list of 'Model Name' values that were ever in the top-n based on 'MMLU price-performance'
top_n = 1
aa_df = aa_df.sort_values(by='Release Date')
ever_top_n_models = set()

unique_dates = aa_df['Release Date'].sort_values().unique()

for date in unique_dates:
    df_up_to_date = aa_df[aa_df['Release Date'] <= date]
    top_n_models = df_up_to_date.nlargest(top_n, 'MMLU price-performance')
    top_n_model_names = top_n_models['Model Name and Date'].tolist()
    ever_top_n_models.update(top_n_model_names)

ever_top_n_list = sorted(ever_top_n_models)
print(ever_top_n_list)

# Plot top-n models over time
fig = px.scatter(aa_df[aa_df['Model Name and Date'].isin(ever_top_n_list)], x='Release Date', y='MMLU price-performance', title='MMLU price-performance over time', hover_data=['Model Name and Date'])
fig.update_layout(yaxis_type='log')
fig.show()

['GPT-3 (2021-11)', 'GPT-3 (2022-09)', 'GPT-3.5 (2022-11)', 'GPT-3.5 Turbo (2023-03)', 'GPT-3.5 Turbo (2023-11)', 'Gemini-1.5-Flash-2024-05 (2024-05)', 'Gemini-1.5-Flash-8B (2024-10)', 'Llama 2-7B (2023-12)', 'Llama-2-Chat-13B (2023-07)', 'Llama-3-Instruct-8B (2024-04)', 'Llama-3.1-Instruct-8B (2024-07)', 'Llama-3.2-Instruct-3B (2024-09)']


## Try fitting a regression to lowest-priced models above a performance lower bound

In [13]:
"""
  - Set a performance lower bound
  - Track the running best (top) model
  - At each point in time (at some resolution)
    - Filter to new models published in this time window
    - Filter to models with performance above the lower bound
    - Check if any new model is cheaper than current best
    - If so, update the current best
    - Record the current best model at this time point
"""
bench = 'MMLU'
performance_lower_bound = 80
performance_upper_bound = 90
ts = pd.date_range(start='2020-01-01', end=pd.Timestamp.today(), freq='MS')
cheapest_models = []
current_best = None

for i, t in enumerate(ts):
    # Get models published in this time window
    benchmark_df = aa_df
    if i > 0:
        prev_t = ts[i-1]
        benchmark_df = benchmark_df[(benchmark_df['Release Date'] >= prev_t) & (benchmark_df['Release Date'] < t)]
    else:
        benchmark_df = benchmark_df[benchmark_df['Release Date'] < t]
        
    # Filter for performance
    benchmark_df = benchmark_df[
        benchmark_df[bench].notna() &
        (benchmark_df[bench] > performance_lower_bound) &
        (benchmark_df[bench] <= performance_upper_bound)
    ]
    
    if not benchmark_df.empty:
        # Find cheapest new model
        new_best = benchmark_df.loc[benchmark_df['USD per 1M Tokens'].idxmin()]
        
        # Update current best if new model is cheaper (or if no current best)
        if current_best is None or new_best['USD per 1M Tokens'] < current_best['USD per 1M Tokens']:
            current_best = new_best
            cheapest_models.append(current_best)
            print(t, current_best['Model Name'], current_best[bench], f"${current_best['USD per 1M Tokens']:.2f}")
        # elif new_best['USD per 1M Tokens'] == current_best['USD per 1M Tokens']:
        #     # Choose the model with the highest performance
        #     if new_best[bench] > current_best[bench]:
        #         current_best = new_best
        #         cheapest_models.append(current_best)
        #         print(t, current_best['Model Name'], current_best[bench], f"${current_best['USD per 1M Tokens']:.2f}")


2023-04-01 00:00:00 GPT-4 86.0 $37.50
2023-12-01 00:00:00 GPT-4 Turbo 87.0 $15.00
2024-06-01 00:00:00 Gemini-1.5-Pro-2024-05 86.0 $2.19
2024-07-01 00:00:00 Claude-3.5-Haiku 81.0 $1.60
2024-08-01 00:00:00 GPT-4o-mini 82.0 $0.26
2025-02-01 00:00:00 Phi 4 85.0 $0.12


In [14]:
cheapest_models_df = pd.DataFrame(cheapest_models)
cheapest_models_df.head()

,Model Name,Release Date,USD per 1M Tokens,Tokens per Second,Prefill Latency (s),Prompt Length (tk),MMLU,GPQA Diamond,MATH-500,HumanEval,LMSys Chatbot Arena ELO,MATH 5,MMLU price-performance,Model Name and Date
4,GPT-4,2023-03-01,37.50,23.6,0.724,1000.0,86.0,33.0,21.0,67.0,1186.0,23.0,2.293333,GPT-4 (2023-03)
10,GPT-4 Turbo,2023-11-01,15.00,39.2,1.246,1000.0,87.0,50.0,74.0,92.0,1256.0,36.0,5.800000,GPT-4 Turbo (2023-11)
26,Gemini-1.5-Pro-2024-05,2024-05-01,2.19,64.8,0.738,1000.0,86.0,46.0,66.0,83.0,1260.0,41.0,39.269406,Gemini-1.5-Pro-2024-05 (2024-05)
31,Claude-3.5-Haiku,2024-06-01,1.60,64.2,0.768,1000.0,81.0,37.0,67.0,87.0,NaN,NaN,50.625000,Claude-3.5-Haiku (2024-06)
38,GPT-4o-mini,2024-07-01,0.26,112.2,0.626,1000.0,82.0,43.0,79.0,88.0,1273.0,48.0,315.384615,GPT-4o-mini (2024-07)


In [15]:
# Fit a line to the data
cheapest_models_df['price'] = cheapest_models_df['USD per 1M Tokens']
cheapest_models_df['log_price'] = np.log10(cheapest_models_df['USD per 1M Tokens'])
cheapest_models_df['date'] = cheapest_models_df['Release Date'].map(lambda x: pd.Timestamp(x).toordinal())
exponential_model = smf.ols('log_price ~ date', data=cheapest_models_df).fit()
print(exponential_model.summary())

linear_model = smf.ols('price ~ date', data=cheapest_models_df).fit()
print(linear_model.summary())

                            OLS Regression Results                            
Dep. Variable:              log_price   R-squared:                       0.897
Model:                            OLS   Adj. R-squared:                  0.871
Method:                 Least Squares   F-statistic:                     34.90
Date:                Thu, 13 Feb 2025   Prob (F-statistic):            0.00411
Time:                        18:32:58   Log-Likelihood:               -0.92738
No. Observations:                   6   AIC:                             5.855
Df Residuals:                       4   BIC:                             5.438
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept   2917.8175    493.886      5.908      0.0

/Users/bencottier/miniconda3/envs/llm-efficiency/lib/python3.12/site-packages/statsmodels/stats/stattools.py:74: ValueWarning:

omni_normtest is not valid with less than 8 observations; 6 samples were given.

/Users/bencottier/miniconda3/envs/llm-efficiency/lib/python3.12/site-packages/statsmodels/stats/stattools.py:74: ValueWarning:

omni_normtest is not valid with less than 8 observations; 6 samples were given.



In [16]:
# Calculate annual rate of decrease
annual_slope = exponential_model.params['date'] * 365  # Convert daily to annual
annual_factor = int(round(10**(-annual_slope)))  # Convert log slope to factor

# Plot the exponential trendline with the data
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=cheapest_models_df['Release Date'],
    y=10**exponential_model.predict(cheapest_models_df['date']),
    mode='lines',
    name=f'Trendline: {annual_factor}x decrease per year',
    line=dict(color='lightgrey', dash='dash')
))
fig.add_trace(go.Scatter(
    x=cheapest_models_df['Release Date'],
    y=cheapest_models_df['USD per 1M Tokens'],
    mode='markers+text',
    name=f'Cheapest models with {performance_lower_bound}-{performance_upper_bound}% on {bench}',
    text=cheapest_models_df['Model Name'],
    textposition='bottom left',
    marker=dict(color='blue')
    # line=dict(shape='hv'),
))
other_models_df = aa_df[(aa_df[bench].notna()) & (aa_df[bench] > performance_lower_bound) & ~(aa_df['Model Name'].isin(cheapest_models_df['Model Name']))]
fig.add_trace(go.Scatter(
    x=other_models_df['Release Date'],
    y=other_models_df['USD per 1M Tokens'],
    mode='markers+text',
    name=f'Other models with {performance_lower_bound}-{performance_upper_bound}% on {bench}',
    text=other_models_df['Model Name'],
    marker=dict(color='lightblue')
))
fig.update_layout(
    title=f'Price of the cheapest model with {performance_lower_bound}-{performance_upper_bound}% on {bench}'
)
fig.update_traces(textposition='bottom left')
fig.update_layout(yaxis_type='log')
fig.update_layout(xaxis_title='Month')
fig.update_layout(yaxis_title='Price in USD per million tokens')
# Lower the lower x limit
min_date = min(cheapest_models_df['Release Date'].min(), other_models_df['Release Date'].min())
max_date = max(cheapest_models_df['Release Date'].max(), other_models_df['Release Date'].max())
fig.update_layout(xaxis_range=[min_date - pd.Timedelta(days=90), max_date + pd.Timedelta(days=30)])
fig.update_layout(
    width=800,
    height=600,
    font=dict(size=10),
    legend=dict(
        yanchor="top",
        y=0.25,
        xanchor="right",
        x=0.4,
        bordercolor="lightgrey",
        borderwidth=1
    )
)
if save:
    os.makedirs(results_dir + 'aa_cheapest_models_examples/', exist_ok=True)
    save_plot(fig, results_dir + 'aa_cheapest_models_examples/', f'aa_cheapest_models_{bench}_above_{performance_lower_bound}_with_trendline')
fig.show()

## Regression on cheapest models above a performance lower bound, one benchmark at a time

In [17]:
performance_step = 10
performance_delta = 10

results_subdir = results_dir + 'aa_cheapest_models_run/'
os.makedirs(results_subdir, exist_ok=True)

results = []
for i, bench in enumerate(benchmarks):
    if i > 0:
        print('\n')
    print(f'{bench}')

    if benchmark_is_mqa[bench]:
        performance_lower_bounds = range(30, 100, performance_step)
    elif bench == 'LMSys Chatbot Arena ELO':
        continue
    else:
        performance_lower_bounds = range(10, 100, performance_step)
    for performance_lower_bound in performance_lower_bounds:
        performance_upper_bound = min(performance_lower_bound + performance_delta, 100)
        print(f'\nPerformance range: {performance_lower_bound}-{performance_upper_bound}%')
        ts = pd.date_range(start='2020-01-01', end=pd.Timestamp.today(), freq='MS')
        cheapest_models = []
        current_best = None

        for i, t in enumerate(ts):
            # Get models published in this time window
            benchmark_df = aa_df
            if i > 0:
                prev_t = ts[i-1]
                benchmark_df = benchmark_df[(benchmark_df['Release Date'] >= prev_t) & (benchmark_df['Release Date'] < t)]
            else:
                benchmark_df = benchmark_df[benchmark_df['Release Date'] < t]
                
            # Filter for performance
            benchmark_df = benchmark_df[
                benchmark_df[bench].notna() &
                (benchmark_df[bench] > performance_lower_bound) &
                (benchmark_df[bench] <= performance_upper_bound)
            ]
            
            if not benchmark_df.empty:
                # Find cheapest new model
                new_best = benchmark_df.loc[benchmark_df['USD per 1M Tokens'].idxmin()]
                
                # Update current best if new model is cheaper (or if no current best)
                if current_best is None or new_best['USD per 1M Tokens'] < current_best['USD per 1M Tokens']:
                    current_best = new_best
                    cheapest_models.append(current_best)
                    print(t, current_best['Model Name'], current_best[bench], f"${current_best['USD per 1M Tokens']:.2f}")

        cheapest_models_df = pd.DataFrame(cheapest_models)
        if len(cheapest_models_df) < min_num_data_points_for_regression:
            print(f'Less than {min_num_data_points_for_regression} models found')
            continue

        # Fit a line to the data
        cheapest_models_df['price'] = cheapest_models_df['USD per 1M Tokens']
        cheapest_models_df['log_price'] = np.log10(cheapest_models_df['USD per 1M Tokens'])
        cheapest_models_df['date'] = cheapest_models_df['Release Date'].map(lambda x: pd.Timestamp(x).toordinal())
        exponential_model = smf.ols('log_price ~ date', data=cheapest_models_df).fit()

        # Calculate annual rate of decrease
        annual_slope = exponential_model.params['date'] * 365  # Convert daily to annual
        annual_factor = int(round(10**(-annual_slope)))  # Convert log slope to factor
        results.append({
            'bench': bench,
            'performance_range': [performance_lower_bound, performance_upper_bound],
            'sample_size': len(cheapest_models_df),
            'start_date': cheapest_models_df['Release Date'].min(),
            'end_date': cheapest_models_df['Release Date'].max(),
            'price_reduction_factor_per_year': annual_factor,
            'r_squared': round(exponential_model.rsquared, 2),
        })

        # Plot the exponential trendline with the data
        fig = go.Figure()
        fig.add_trace(go.Scatter(
            x=cheapest_models_df['Release Date'],
            y=10**exponential_model.predict(cheapest_models_df['date']),
            mode='lines',
            name=f'Trendline: {annual_factor}x decrease per year',
            line=dict(color='lightgrey', dash='dash')
        ))
        fig.add_trace(go.Scatter(
            x=cheapest_models_df['Release Date'],
            y=cheapest_models_df['USD per 1M Tokens'],
            mode='markers+text',
            name=f'Cheapest models with {performance_lower_bound}-{performance_upper_bound}% on {bench}',
            text=cheapest_models_df['Model Name'],
            textposition='bottom left',
            marker=dict(color='blue')
            # line=dict(shape='hv'),
        ))
        other_models_df = aa_df[
            aa_df[bench].notna() &
            (aa_df[bench] > performance_lower_bound) &
            (aa_df[bench] <= performance_upper_bound) &
            ~(aa_df['Model Name'].isin(cheapest_models_df['Model Name']))
        ]
        print(cheapest_models_df['Model Name'].to_list())
        print(other_models_df)
        fig.add_trace(go.Scatter(
            x=other_models_df['Release Date'],
            y=other_models_df['USD per 1M Tokens'],
            mode='markers',
            name=f'Other models with {performance_lower_bound}-{performance_upper_bound}% on {bench}',
            text=other_models_df['Model Name'],
            marker=dict(color='lightblue')
        ))
        fig.update_layout(
            title=f'The cheapest model with {performance_lower_bound}-{performance_upper_bound}% on {bench} has become {annual_factor}x cheaper per year'
        )
        fig.update_traces(textposition='bottom left')
        fig.update_layout(yaxis_type='log')
        fig.update_layout(xaxis_title='Month')
        fig.update_layout(yaxis_title='Price in USD per million tokens')
        # Lower the lower x limit
        min_date = min(cheapest_models_df['Release Date'].min(), other_models_df['Release Date'].min())
        max_date = max(cheapest_models_df['Release Date'].max(), other_models_df['Release Date'].max())
        fig.update_layout(xaxis_range=[min_date - pd.Timedelta(days=90), max_date + pd.Timedelta(days=30)])
        fig.update_layout(
            width=800,
            height=600,
            font=dict(size=10),
            legend=dict(
                yanchor="top",
                y=0.99,
                xanchor="right",
                x=0.99,
                bordercolor="lightgrey",
                borderwidth=1
            )
        )
        if save:
            save_plot(
                fig,
                results_subdir,
                f'aa_cheapest_models_{bench}_{performance_lower_bound}-{performance_upper_bound}_with_trendline',
                extensions=['png'],
            )

cheapest_model_results_df = pd.DataFrame(results)
cheapest_model_results_df.to_csv(results_subdir + 'cheapest_model_results.csv', index=False)

# Create a summary DataFrame
summary_data = []
for bench in benchmarks:
    if bench == 'LMSys Chatbot Arena ELO':
        continue
    bench_results = cheapest_model_results_df[cheapest_model_results_df['bench'] == bench]
    # Get all performance lower bounds used for this benchmark
    perf_bounds = list(bench_results['performance_range'])
    
    # Calculate geometric mean of price reduction factors
    price_factors = bench_results['price_reduction_factor_per_year'].dropna()
    geomean = np.exp(np.mean(np.log(price_factors))) if len(price_factors) > 0 else np.nan
    
    # Get range of price reduction factors
    factor_range = [price_factors.min(), price_factors.max()] if len(price_factors) > 0 else []
    
    summary_data.append({
        'bench': bench,
        'performance_range': perf_bounds,
        'price_reduction_factor_per_year_geomean': round(geomean),
        'price_reduction_factor_per_year_range': factor_range
    })

cheapest_model_summary_df = pd.DataFrame(summary_data)
cheapest_model_summary_df.to_csv(results_subdir + 'cheapest_model_summary.csv', index=False)

MMLU

Performance range: 30-40%
2024-10-01 00:00:00 Llama-3.2-Instruct-1B 35.0 $0.05
Less than 4 models found

Performance range: 40-50%
2021-12-01 00:00:00 GPT-3 43.9 $60.00
2022-10-01 00:00:00 GPT-3 43.9 $20.00
2023-08-01 00:00:00 Llama-2-Chat-13B 45.0 $0.56
2024-01-01 00:00:00 Llama 2-7B 45.3 $0.20
2024-05-01 00:00:00 Llama 2-7B 45.3 $0.13
['GPT-3', 'GPT-3', 'Llama-2-Chat-13B', 'Llama 2-7B', 'Llama 2-7B']
Empty DataFrame
Columns: [Model Name, Release Date, USD per 1M Tokens, Tokens per Second, Prefill Latency (s), Prompt Length (tk), MMLU, GPQA Diamond, MATH-500, HumanEval, LMSys Chatbot Arena ELO, MATH 5, MMLU price-performance, Model Name and Date]
Index: []

Performance range: 50-60%
2024-01-01 00:00:00 Llama 2-13B 54.8 $0.35
2024-05-01 00:00:00 Llama 2-13B 54.8 $0.22
Less than 4 models found

Performance range: 60-70%
2022-12-01 00:00:00 GPT-3.5 64.8 $20.00
2023-04-01 00:00:00 GPT-3.5 Turbo 68.0 $2.00
2023-12-01 00:00:00 GPT-3.5 Turbo 68.0 $0.75
2024-05-01 00:00:00 Llama-3-Instr

### Visualize all at once

In [18]:
performance_step = 10
performance_delta = 10
models_ever_in_cheapest = set()
cheapest_models_lookup = defaultdict(lambda: {})
regression_models_lookup = defaultdict(lambda: {})
for i, bench in enumerate(benchmarks):
    if i > 0:
        print('\n')
    print(f'{bench}')

    if benchmark_is_mqa[bench]:
        performance_lower_bounds = range(30, 100, performance_step)
    elif bench == 'LMSys Chatbot Arena ELO':
        continue
    else:
        performance_lower_bounds = range(10, 100, performance_step)
    
    for performance_lower_bound in performance_lower_bounds:
        performance_upper_bound = min(performance_lower_bound + performance_delta, 100)
        print(f'\nPerformance range: {performance_lower_bound}-{performance_upper_bound}%')
        ts = pd.date_range(start='2020-01-01', end=pd.Timestamp.today(), freq='MS')
        cheapest_models = []
        current_best = None

        for i, t in enumerate(ts):
            # Get models published in this time window
            benchmark_df = aa_df
            if i > 0:
                prev_t = ts[i-1]
                benchmark_df = benchmark_df[(benchmark_df['Release Date'] >= prev_t) & (benchmark_df['Release Date'] < t)]
            else:
                benchmark_df = benchmark_df[benchmark_df['Release Date'] < t]
                
            # Filter for performance
            benchmark_df = benchmark_df[
                benchmark_df[bench].notna() &
                (benchmark_df[bench] > performance_lower_bound) &
                (benchmark_df[bench] <= performance_upper_bound)
            ]
            
            if not benchmark_df.empty:
                # Find cheapest new model
                new_best = benchmark_df.loc[benchmark_df['USD per 1M Tokens'].idxmin()]
                
                # Update current best if new model is cheaper (or if no current best)
                if current_best is None or new_best['USD per 1M Tokens'] < current_best['USD per 1M Tokens']:
                    current_best = new_best
                    cheapest_models.append(current_best)
                    print(t, current_best['Model Name'], current_best[bench], f"${current_best['USD per 1M Tokens']:.2f}")

        cheapest_models_df = pd.DataFrame(cheapest_models)
        if len(cheapest_models_df) > 0:
            models_ever_in_cheapest.update(cheapest_models_df['Model Name'])
        cheapest_models_lookup[bench][str([performance_lower_bound, performance_upper_bound])] = cheapest_models_df

        # Fit a line to the data
        if len(cheapest_models_df) > 1:
            cheapest_models_df['price'] = cheapest_models_df['USD per 1M Tokens']
            cheapest_models_df['log_price'] = np.log10(cheapest_models_df['USD per 1M Tokens'])
            cheapest_models_df['date'] = cheapest_models_df['Release Date'].map(lambda x: pd.Timestamp(x).toordinal())
            exponential_model = smf.ols('log_price ~ date', data=cheapest_models_df).fit()
            regression_models_lookup[bench][str([performance_lower_bound, performance_upper_bound])] = exponential_model
        else:
            regression_models_lookup[bench][str([performance_lower_bound, performance_upper_bound])] = None

MMLU

Performance range: 30-40%
2024-10-01 00:00:00 Llama-3.2-Instruct-1B 35.0 $0.05

Performance range: 40-50%
2021-12-01 00:00:00 GPT-3 43.9 $60.00
2022-10-01 00:00:00 GPT-3 43.9 $20.00
2023-08-01 00:00:00 Llama-2-Chat-13B 45.0 $0.56
2024-01-01 00:00:00 Llama 2-7B 45.3 $0.20
2024-05-01 00:00:00 Llama 2-7B 45.3 $0.13

Performance range: 50-60%
2024-01-01 00:00:00 Llama 2-13B 54.8 $0.35
2024-05-01 00:00:00 Llama 2-13B 54.8 $0.22

Performance range: 60-70%
2022-12-01 00:00:00 GPT-3.5 64.8 $20.00
2023-04-01 00:00:00 GPT-3.5 Turbo 68.0 $2.00
2023-12-01 00:00:00 GPT-3.5 Turbo 68.0 $0.75
2024-05-01 00:00:00 Llama-3-Instruct-8B 64.0 $0.15
2024-07-01 00:00:00 Mistral-NeMo 66.0 $0.13
2024-10-01 00:00:00 Llama-3.2-Instruct-3B 64.0 $0.08

Performance range: 70-80%
2024-04-01 00:00:00 Claude-3-Haiku 71.0 $0.50
2024-06-01 00:00:00 Gemini-1.5-Flash-2024-05 79.0 $0.13
2024-08-01 00:00:00 Llama-3.1-Instruct-8B 71.0 $0.10
2024-11-01 00:00:00 Gemini-1.5-Flash-8B 75.0 $0.07

Performance range: 80-90%
20

In [19]:
print(models_ever_in_cheapest)

{'GPT-4o-2024-08', 'o1-mini', 'GPT-3', 'GPT-4o-mini', 'Claude-3.5-Sonnet-2024-06', 'Llama-3.1-Instruct-70B', 'GPT-3.5 Turbo', 'Gemma-2-9B', 'Gemini-1.5-Flash-2024-05', 'GPT-3.5', 'Mistral-Small-2024-02', 'o1', 'Llama 2-13B', 'Llama-3.2-Instruct-3B', 'Gemini-1.5-Flash-8B', 'Mistral-Large-2024-02', 'Llama-2-Chat-7B', 'Llama-3.1-Instruct-8B', 'Llama-3-Instruct-70B', 'Llama-3.2-Instruct-1B', 'Llama 2-7B', 'Claude-3.5-Haiku', 'Llama-2-Chat-13B', 'DeepSeek-R1', 'Claude-3-Opus', 'Claude-3-Haiku', 'Phi 4', 'Gemini-1.5-Flash-2024-09', 'GPT-4 Turbo', 'DeepSeek-V3', 'Gemini-1.5-Pro-2024-09', 'Gemini-1.5-Pro-2024-05', 'Llama-3-Instruct-8B', 'Mistral-Large-2-2024-06', 'GPT-4', 'GPT-4o-2024-05', 'Mistral-NeMo'}


In [20]:
print(set(aa_df['Model Name']) - models_ever_in_cheapest)

{'Mistral Small 3', 'Claude Instant', 'Gemini 1.0 Pro', 'o3-mini', 'Cohere Command', 'Gemini 2.0 Flash', 'Command R+', 'Cohere Command Light', 'Claude-3.5-Sonnet-2024-10', 'Command R', 'Mistral-8x22', 'o1-preview', 'Jamba 1.5-Large', 'Claude-3-Sonnet', 'Llama-3.1-Instruct-405B', 'Mistral 7B', 'Gemma-2-27B', 'GPT-4o-2024-11', 'Reka Core', 'DeepSeek-Coder-V2 236B', 'Llama 2-70B Chat', 'Claude 2.1', 'Mistral-Large-2-2024-11', 'Claude 2'}


In [21]:
len(set(aa_df['Model Name'])), len(models_ever_in_cheapest)

(61, 37)

In [22]:
cheapest_models_lookup['MMLU']['[80, 90]']

,Model Name,Release Date,USD per 1M Tokens,Tokens per Second,Prefill Latency (s),Prompt Length (tk),MMLU,GPQA Diamond,MATH-500,HumanEval,LMSys Chatbot Arena ELO,MATH 5,MMLU price-performance,Model Name and Date,price,log_price,date
4,GPT-4,2023-03-01,37.5000,23.6,0.724,1000.0,86.0,33.0,21.0,67.0,1186.0,23.0,2.293333,GPT-4 (2023-03),37.5000,1.574031,738580
10,GPT-4 Turbo,2023-11-01,15.0000,39.2,1.246,1000.0,87.0,50.0,74.0,92.0,1256.0,36.0,5.800000,GPT-4 Turbo (2023-11),15.0000,1.176091,738825
26,Gemini-1.5-Pro-2024-05,2024-05-01,2.1900,64.8,0.738,1000.0,86.0,46.0,66.0,83.0,1260.0,41.0,39.269406,Gemini-1.5-Pro-2024-05 (2024-05),2.1900,0.340444,739007
31,Claude-3.5-Haiku,2024-06-01,1.6000,64.2,0.768,1000.0,81.0,37.0,67.0,87.0,NaN,NaN,50.625000,Claude-3.5-Haiku (2024-06),1.6000,0.204120,739038
38,GPT-4o-mini,2024-07-01,0.2600,112.2,0.626,1000.0,82.0,43.0,79.0,88.0,1273.0,48.0,315.384615,GPT-4o-mini (2024-07),0.2600,-0.585027,739068
65,Phi 4,2025-01-01,0.1225,NaN,NaN,NaN,85.0,53.0,81.0,87.0,NaN,65.0,693.877551,Phi 4 (2025-01),0.1225,-0.911864,739252


In [23]:

fig = go.Figure()
fig.add_trace(go.Scatter(
    x=aa_df['Release Date'],
    y=aa_df['USD per 1M Tokens'],
    mode='markers',
    name=f'All models',
    text=aa_df['Model Name'],
    marker=dict(color='lightblue')
))
for bench in benchmarks:
    if benchmark_is_mqa[bench]:
        performance_lower_bounds = range(30, 100, performance_step)
    elif bench == 'LMSys Chatbot Arena ELO':
        continue
    else:
        performance_lower_bounds = range(10, 100, performance_step)
    
    for performance_lower_bound in performance_lower_bounds:
        performance_upper_bound = min(performance_lower_bound + performance_delta, 100)

        performance_range_str = str([performance_lower_bound, performance_upper_bound])

        cheapest_models_df = cheapest_models_lookup[bench][performance_range_str]

        if len(cheapest_models_df) == 0:
            continue

        fig.add_trace(go.Scatter(
            x=cheapest_models_df['Release Date'],
            y=cheapest_models_df['USD per 1M Tokens'],
            mode='markers+text',
            name=f'Cheapest models with {performance_lower_bound}-{performance_upper_bound}% on {bench}',
            text=cheapest_models_df['Model Name'],
            textposition='bottom left',
            # marker=dict(color='blue'),
            visible="legendonly",  # Hide this trace by default
            legendgroup=f'{bench}_{performance_range_str}',
        ))

        exponential_model = regression_models_lookup[bench][performance_range_str]
        if exponential_model is None:
            continue
        
        annual_slope = exponential_model.params['date'] * 365  # Convert daily to annual
        annual_factor = int(round(10**(-annual_slope)))  # Convert log slope to factor
        # Plot the exponential trendline with the data
        fig.add_trace(go.Scatter(
            x=cheapest_models_df['Release Date'],
            y=10**exponential_model.predict(cheapest_models_df['date']),
            mode='lines',
            name=f'{annual_factor}x decrease per year',
            visible="legendonly",
            legendgroup=f'{bench}_{performance_range_str}',
            line=dict(color='lightgrey', dash='dash')
        ))
fig.update_layout(
    title=f'API price of LLMs over time'
)
fig.update_traces(textposition='bottom left')
fig.update_layout(yaxis_type='log')
fig.update_layout(xaxis_title='Month')
fig.update_layout(yaxis_title='Price in USD per million tokens')
# Lower the lower x limit
fig.update_layout(xaxis_range=[aa_df['Release Date'].min() - pd.Timedelta(days=90), aa_df['Release Date'].max() + pd.Timedelta(days=30)])
fig.update_layout(
    width=1000,
    height=600,
    font=dict(size=10),
    # legend=dict(
    #     yanchor="top",
    #     y=0.99,
    #     xanchor="right",
    #     x=0.99,
    #     bordercolor="lightgrey",
    #     borderwidth=1
    # )
)

if save:
    save_plot(fig, results_subdir, 'aa_cheapest_models_all_interactive', extensions=['html'])

fig.show()

## Regression on cheapest models above a performance lower bound, average benchmark

In [24]:
df = aa_df.copy()
# Rename benchmark columns to all lowercase, underscore separated
new_bench_cols = [col.lower().replace(' ', '_').replace('-', '_') for col in benchmarks]
df.rename(columns={bench: new_bench_cols[i] for i, bench in enumerate(benchmarks)}, inplace=True)
df = df.dropna(subset=new_bench_cols)
df['avg_bench'] = df[new_bench_cols].mean(axis=1)
df[['Model Name', 'Release Date', 'avg_bench']].sort_values(by='avg_bench', ascending=False).head(10)

,Model Name,Release Date,avg_bench
70,Gemini 2.0 Flash,2025-02-01,83.0
48,Gemini-1.5-Pro-2024-09,2024-09-01,78.0
60,Claude-3.5-Sonnet-2024-10,2024-10-01,74.4
65,Phi 4,2025-01-01,74.2
39,GPT-4o-2024-08,2024-08-01,72.0
28,GPT-4o-2024-05,2024-05-01,71.6
32,Claude-3.5-Sonnet-2024-06,2024-06-01,70.2
61,GPT-4o-2024-11,2024-11-01,69.8
49,Gemini-1.5-Flash-2024-09,2024-09-01,68.8
38,GPT-4o-mini,2024-07-01,68.0


In [25]:
performance_step = 10
performance_delta = 10
performance_lower_bounds = range(10, 100, performance_step)

results_subdir = results_dir + 'aa_cheapest_models_avg_benchmark_run/'
os.makedirs(results_subdir, exist_ok=True)

bench = 'avg_bench'
results = []
for performance_lower_bound in performance_lower_bounds:
    performance_upper_bound = min(performance_lower_bound + performance_delta, 100)
    print(f'\nPerformance range: {performance_lower_bound}-{performance_upper_bound}%')
    ts = pd.date_range(start='2020-01-01', end=pd.Timestamp.today(), freq='MS')
    cheapest_models = []
    current_best = None

    for i, t in enumerate(ts):
        # Get models published in this time window
        benchmark_df = df
        if i > 0:
            prev_t = ts[i-1]
            benchmark_df = benchmark_df[(benchmark_df['Release Date'] >= prev_t) & (benchmark_df['Release Date'] < t)]
        else:
            benchmark_df = benchmark_df[benchmark_df['Release Date'] < t]
            
        # Filter for performance
        benchmark_df = benchmark_df[
            benchmark_df[bench].notna() &
            (benchmark_df[bench] > performance_lower_bound) &
            (benchmark_df[bench] <= performance_upper_bound)
        ]
        
        if not benchmark_df.empty:
            # Find cheapest new model
            new_best = benchmark_df.loc[benchmark_df['USD per 1M Tokens'].idxmin()]
            
            # Update current best if new model is cheaper (or if no current best)
            if current_best is None or new_best['USD per 1M Tokens'] < current_best['USD per 1M Tokens']:
                current_best = new_best
                cheapest_models.append(current_best)
                print(t, current_best['Model Name'], current_best[bench], f"${current_best['USD per 1M Tokens']:.2f}")

    cheapest_models_df = pd.DataFrame(cheapest_models)
    if len(cheapest_models_df) < min_num_data_points_for_regression:
        print(f'Less than {min_num_data_points_for_regression} models found')
        continue

    # Fit a line to the data
    cheapest_models_df['price'] = cheapest_models_df['USD per 1M Tokens']
    cheapest_models_df['log_price'] = np.log10(cheapest_models_df['USD per 1M Tokens'])
    cheapest_models_df['date'] = cheapest_models_df['Release Date'].map(lambda x: pd.Timestamp(x).toordinal())
    exponential_model = smf.ols('log_price ~ date', data=cheapest_models_df).fit()

    # Calculate annual rate of decrease
    annual_slope = exponential_model.params['date'] * 365  # Convert daily to annual
    annual_factor = int(round(10**(-annual_slope)))  # Convert log slope to factor
    results.append({
        'bench': bench,
        'performance_range': [performance_lower_bound, performance_upper_bound],
        'sample_size': len(cheapest_models_df),
        'start_date': cheapest_models_df['Release Date'].min(),
        'end_date': cheapest_models_df['Release Date'].max(),
        'price_reduction_factor_per_year': annual_factor,
        'r_squared': round(exponential_model.rsquared, 2),
    })

    # Plot the exponential trendline with the data
    fig = go.Figure()
    fig.add_trace(go.Scatter(
        x=cheapest_models_df['Release Date'],
        y=10**exponential_model.predict(cheapest_models_df['date']),
        mode='lines',
        name=f'Trendline: {annual_factor}x decrease per year',
        line=dict(color='lightgrey', dash='dash')
    ))
    fig.add_trace(go.Scatter(
        x=cheapest_models_df['Release Date'],
        y=cheapest_models_df['USD per 1M Tokens'],
        mode='markers+text',
        name=f'Cheapest models with {performance_lower_bound}-{performance_upper_bound}% on {bench}',
        text=cheapest_models_df['Model Name'],
        textposition='bottom left',
        marker=dict(color='blue')
        # line=dict(shape='hv'),
    ))
    other_models_df = df[(
        df[bench].notna()) &
        (df[bench] > performance_lower_bound) &
        (df[bench] <= performance_upper_bound) &
        ~(df['Model Name'].isin(cheapest_models_df['Model Name'])
    )]
    fig.add_trace(go.Scatter(
        x=other_models_df['Release Date'],
        y=other_models_df['USD per 1M Tokens'],
        mode='markers',
        name=f'Other models with {performance_lower_bound}-{performance_upper_bound}% on {bench}',
        text=other_models_df['Model Name'],
        marker=dict(color='lightblue')
    ))
    fig.update_layout(
        title=f'The cheapest model with {performance_lower_bound}-{performance_upper_bound}% on {bench} has become {annual_factor}x cheaper per year'
    )
    fig.update_traces(textposition='bottom left')
    fig.update_layout(yaxis_type='log')
    fig.update_layout(xaxis_title='Month')
    fig.update_layout(yaxis_title='Price in USD per million tokens')
    # Lower the lower x limit
    min_date = min(cheapest_models_df['Release Date'].min(), other_models_df['Release Date'].min())
    max_date = max(cheapest_models_df['Release Date'].max(), other_models_df['Release Date'].max())
    fig.update_layout(xaxis_range=[min_date - pd.Timedelta(days=90), max_date + pd.Timedelta(days=30)])
    fig.update_layout(
        width=800,
        height=600,
        font=dict(size=10),
        legend=dict(
            yanchor="top",
            y=0.99,
            xanchor="right",
            x=0.99,
            bordercolor="lightgrey",
            borderwidth=1
        )
    )
    if save:
        save_plot(
            fig,
            results_subdir,
            f'aa_cheapest_models_{bench}_{performance_lower_bound}-{performance_upper_bound}_with_trendline',
            extensions=['png'],
        )

cheapest_model_results_df = pd.DataFrame(results)
cheapest_model_results_df.to_csv(results_subdir + 'cheapest_model_results.csv', index=False)

# Create a summary DataFrame
summary_data = []
bench_results = cheapest_model_results_df[cheapest_model_results_df['bench'] == bench]
# Get all performance lower bounds used for this benchmark
perf_bounds = list(bench_results['performance_range'])

# Calculate geometric mean of price reduction factors
price_factors = bench_results['price_reduction_factor_per_year'].dropna()
geomean = np.exp(np.mean(np.log(price_factors))) if len(price_factors) > 0 else np.nan

# Get range of price reduction factors
factor_range = [price_factors.min(), price_factors.max()] if len(price_factors) > 0 else []

summary_data.append({
    'bench': bench,
    'performance_range': perf_bounds,
    'price_reduction_factor_per_year_geomean': round(geomean),
    'price_reduction_factor_per_year_range': factor_range
})

cheapest_model_avg_benchmark_summary_df = pd.DataFrame(summary_data)
cheapest_model_avg_benchmark_summary_df.to_csv(results_subdir + 'cheapest_model_summary.csv', index=False)


Performance range: 10-20%
Less than 4 models found

Performance range: 20-30%
Less than 4 models found

Performance range: 30-40%
2024-05-01 00:00:00 Llama-3-Instruct-8B 37.4 $0.15
Less than 4 models found

Performance range: 40-50%
2023-04-01 00:00:00 GPT-4 46.0 $37.50
2023-07-01 00:00:00 GPT-3.5 Turbo 45.36 $3.25
2023-12-01 00:00:00 GPT-3.5 Turbo 45.6 $0.75
2024-04-01 00:00:00 Claude-3-Haiku 46.6 $0.50
2024-07-01 00:00:00 Mistral-NeMo 44.8 $0.13
2024-08-01 00:00:00 Llama-3.1-Instruct-8B 47.4 $0.10

Performance range: 50-60%
2024-05-01 00:00:00 Mistral-8x22 53.2 $1.20
2024-07-01 00:00:00 Gemma-2-27B 53.8 $0.26
Less than 4 models found

Performance range: 60-70%
2023-12-01 00:00:00 GPT-4 Turbo 67.8 $15.00
2024-06-01 00:00:00 Gemini-1.5-Pro-2024-05 64.4 $2.19
2024-08-01 00:00:00 GPT-4o-mini 68.0 $0.26
2024-10-01 00:00:00 Gemini-1.5-Flash-2024-09 68.8 $0.13

Performance range: 70-80%
2024-06-01 00:00:00 GPT-4o-2024-05 71.6 $7.50
2024-07-01 00:00:00 Claude-3.5-Sonnet-2024-06 70.2 $6.00
2

## Regression on top models by price-performance, one benchmark at a time

In [26]:
# One-off analysis for GPQA benchmark
results_subdir = f'aa_top_{top_n}_price_performance_run/'
os.makedirs(results_dir + results_subdir, exist_ok=True)
bench = 'GPQA Diamond'
benchmark_df = aa_df.copy()
benchmark_df = benchmark_df.dropna(subset=[bench])
if benchmark_is_mqa[bench]:
    # Filter for above-random-chance baseline
    # Use 30% rather than 25%, for signififance
    benchmark_df = benchmark_df[benchmark_df[bench] > 30]
benchmark_df = benchmark_df.sort_values(by='Release Date')
benchmark_df[f'{bench} price-performance'] = benchmark_df[bench] / benchmark_df['USD per 1M Tokens']

# Find the rolling top-n models
unique_dates = benchmark_df['Release Date'].unique()
ever_top_n_models = set()
for date in unique_dates:
    df_up_to_date = benchmark_df[benchmark_df['Release Date'] <= date]
    top_n_models = df_up_to_date.nlargest(1, f'{bench} price-performance')  # top_n = 1
    top_n_model_names = top_n_models['Model Name and Date'].tolist()
    ever_top_n_models.update(top_n_model_names)
ever_top_n_list = sorted(ever_top_n_models)
benchmark_df = benchmark_df[benchmark_df['Model Name and Date'].isin(ever_top_n_list)]

if len(benchmark_df) >= min_num_data_points_for_regression:
    # Fit a line to the data
    benchmark_df['price'] = benchmark_df['USD per 1M Tokens']
    benchmark_df['log_price'] = np.log10(benchmark_df['USD per 1M Tokens'])
    benchmark_df['bench'] = benchmark_df[bench]
    benchmark_df['date'] = benchmark_df['Release Date'].map(lambda x: pd.Timestamp(x).toordinal())
    exponential_model = smf.ols('log_price ~ date + bench', data=benchmark_df).fit()

    # Calculate annual rate of decrease
    annual_slope = exponential_model.params['date'] * 365  # Convert daily to annual
    annual_factor = int(round(10**(-annual_slope)))  # Convert log slope to factor
    annual_slope_ci = exponential_model.conf_int(alpha=0.1).loc['date']
    annual_slope_ci_high = int(round(10**(-annual_slope_ci[0] * 365)))
    annual_slope_ci_low = int(round(10**(-annual_slope_ci[1] * 365)))
    
    print(f"Price reduction factor per year (mean): {annual_factor}")
    print(f"90% CI: [{annual_slope_ci_low}, {annual_slope_ci_high}]")
    print(f"Sample size: {len(benchmark_df)}")
    print(f"Date range: {benchmark_df['Release Date'].min()} to {benchmark_df['Release Date'].max()}")

    prediction_dates = pd.date_range(start=benchmark_df['Release Date'].min(), end=benchmark_df['Release Date'].max(), freq='MS')
    reference_model = 'GPT-4'
    reference_performance = aa_df[aa_df['Model Name'] == reference_model][bench].iloc[0]
    prediction_df = pd.DataFrame({
        'date': [pd.Timestamp(d).toordinal() for d in prediction_dates],  # Convert to ordinal dates
        'bench': [reference_performance] * len(prediction_dates)
    })
    prediction_df['log_price'] = exponential_model.predict(prediction_df)
    prediction_df['price'] = 10**prediction_df['log_price']

    fig = go.Figure()
    fig.add_trace(go.Scatter(
        x=prediction_dates,
        y=prediction_df['price'],
        mode='lines',
        name=f'Regression: {annual_factor}x decrease per year at {reference_model} performance',
        line=dict(color='lightgrey', dash='dash')
    ))
    fig.add_trace(go.Scatter(
        x=benchmark_df['Release Date'],
        y=benchmark_df['USD per 1M Tokens'],
        mode='markers+text',
        name='Model with the best price-performance at the time',
        text=benchmark_df['Model Name'],
        textposition='bottom left',
        marker=dict(color='blue'),
    ))
    other_models_df = aa_df[(
        aa_df[bench].notna()) &
        ~(aa_df['Model Name'].isin(benchmark_df['Model Name'])
    )]
    fig.add_trace(go.Scatter(
        x=other_models_df['Release Date'],
        y=other_models_df['USD per 1M Tokens'],
        mode='markers',
        name=f'Other models',
        text=other_models_df['Model Name'],
        marker=dict(color='lightblue')
    ))
    fig.update_layout(
        title=f'Price of models with the best price-performance on {bench}'
    )
    fig.update_traces(textposition='bottom left')
    fig.update_layout(yaxis_type='log')
    fig.update_layout(xaxis_title='Month')
    fig.update_layout(yaxis_title='Price in USD per million tokens')
    # Lower the lower x limit
    fig.update_layout(xaxis_range=[benchmark_df['Release Date'].min() - pd.Timedelta(days=90), benchmark_df['Release Date'].max()+pd.Timedelta(days=30)])
    fig.update_layout(
        width=800,
        height=600,
        font=dict(size=10),
        legend=dict(
            yanchor="top",
            y=0.99,
            xanchor="right",
            x=0.99,
            bordercolor="lightgrey",
            borderwidth=1
        )
    )
    # if save:
    #     save_plot(
    #         fig,
    #         results_dir + results_subdir,
    #         f'top_{top_n}_price_performance_{bench}_with_trendline',
    #         extensions=['png'],
    #     )
    fig.show()
else:
    print(f'Less than {min_num_data_points_for_regression} models found for {bench}')


Price reduction factor per year (mean): 88
90% CI: [21, 367]
Sample size: 7
Date range: 2023-03-01 00:00:00 to 2025-01-01 00:00:00


In [27]:
top_n = 1
results_subdir = f'aa_top_{top_n}_price_performance_run/'
os.makedirs(results_dir + results_subdir, exist_ok=True)

# Open log file
results = []
for bench in benchmarks:
    benchmark_df = aa_df.copy()
    benchmark_df = benchmark_df.dropna(subset=[bench])
    if benchmark_is_mqa[bench]:
        # Filter for above-random-chance baseline
        # Use 30% rather than 25%, for signififance
        benchmark_df = benchmark_df[benchmark_df[bench] > 30]
    benchmark_df = benchmark_df.sort_values(by='Release Date')
    benchmark_df[f'{bench} price-performance'] = benchmark_df[bench] / benchmark_df['USD per 1M Tokens']

    # Find the rolling top-n models
    unique_dates = benchmark_df['Release Date'].unique()
    ever_top_n_models = set()
    for date in unique_dates:
        df_up_to_date = benchmark_df[benchmark_df['Release Date'] <= date]
        top_n_models = df_up_to_date.nlargest(top_n, f'{bench} price-performance')
        top_n_model_names = top_n_models['Model Name and Date'].tolist()
        ever_top_n_models.update(top_n_model_names)
    ever_top_n_list = sorted(ever_top_n_models)
    benchmark_df = benchmark_df[benchmark_df['Model Name and Date'].isin(ever_top_n_list)]

    if len(benchmark_df) < min_num_data_points_for_regression:
        print(f'Less than {min_num_data_points_for_regression} models found for {bench}')
        continue

    # Fit a line to the data
    benchmark_df['price'] = benchmark_df['USD per 1M Tokens']
    benchmark_df['log_price'] = np.log10(benchmark_df['USD per 1M Tokens'])
    benchmark_df['bench'] = benchmark_df[bench]
    benchmark_df['date'] = benchmark_df['Release Date'].map(lambda x: pd.Timestamp(x).toordinal())
    exponential_model = smf.ols('log_price ~ date + bench', data=benchmark_df).fit()

    # Calculate annual rate of decrease
    annual_slope = exponential_model.params['date'] * 365  # Convert daily to annual
    annual_factor = int(round(10**(-annual_slope)))  # Convert log slope to factor
    annual_slope_ci = exponential_model.conf_int(alpha=0.1).loc['date']
    annual_slope_ci_high = int(round(10**(-annual_slope_ci[0] * 365)))
    annual_slope_ci_low = int(round(10**(-annual_slope_ci[1] * 365)))
    results.append({
        'bench': bench,
        'sample_size': len(benchmark_df),
        'start_date': benchmark_df['Release Date'].min(),
        'end_date': benchmark_df['Release Date'].max(),
        'price_reduction_factor_per_year_mean': annual_factor,
        'price_reduction_factor_per_year_ci': [annual_slope_ci_low, annual_slope_ci_high],
        'r_squared': round(exponential_model.rsquared, 2),
    })

    # Plot the exponential trendline with the data

    # Use GPT-4 as the reference point
    prediction_dates = pd.date_range(start=benchmark_df['Release Date'].min(), end=benchmark_df['Release Date'].max(), freq='MS')
    reference_model = 'GPT-4'
    reference_performance = aa_df[aa_df['Model Name'] == reference_model][bench].iloc[0]
    prediction_df = pd.DataFrame({
        'date': [pd.Timestamp(d).toordinal() for d in prediction_dates],  # Convert to ordinal dates
        'bench': [reference_performance] * len(prediction_dates)
    })
    prediction_df['log_price'] = exponential_model.predict(prediction_df)
    prediction_df['price'] = 10**prediction_df['log_price']

    fig = go.Figure()
    fig.add_trace(go.Scatter(
        x=prediction_dates,
        y=prediction_df['price'],
        mode='lines',
        name=f'Regression: {annual_factor}x decrease per year at {reference_model} performance',
        line=dict(color='lightgrey', dash='dash')
    ))
    fig.add_trace(go.Scatter(
        x=benchmark_df['Release Date'],
        y=benchmark_df['USD per 1M Tokens'],
        mode='markers+text',
        name='Model with the best price-performance at the time',
        text=benchmark_df['Model Name'],
        textposition='bottom left',
        marker=dict(color='blue'),
    ))
    other_models_df = aa_df[(
        aa_df[bench].notna()) &
        ~(aa_df['Model Name'].isin(benchmark_df['Model Name'])
    )]
    fig.add_trace(go.Scatter(
        x=other_models_df['Release Date'],
        y=other_models_df['USD per 1M Tokens'],
        mode='markers',
        name=f'Other models',
        text=other_models_df['Model Name'],
        marker=dict(color='lightblue')
    ))
    fig.update_layout(
        title=f'Models with the best price-performance on {bench} get {annual_factor}x cheaper per year'
    )
    fig.update_traces(textposition='bottom left')
    fig.update_layout(yaxis_type='log')
    fig.update_layout(xaxis_title='Month')
    fig.update_layout(yaxis_title='Price in USD per million tokens')
    # Lower the lower x limit
    fig.update_layout(xaxis_range=[benchmark_df['Release Date'].min() - pd.Timedelta(days=90), benchmark_df['Release Date'].max()+pd.Timedelta(days=30)])
    fig.update_layout(
        width=800,
        height=600,
        font=dict(size=10),
        legend=dict(
            yanchor="top",
            y=0.99,
            xanchor="right",
            x=0.99,
            bordercolor="lightgrey",
            borderwidth=1
        )
    )
    if save:
        save_plot(
            fig,
            results_dir + results_subdir,
            f'top_{top_n}_price_performance_{bench}_with_trendline',
            extensions=['png'],
        )

top_n_price_performance_results_df = pd.DataFrame(results)
top_n_price_performance_results_df.to_csv(results_dir + results_subdir + 'top_n_price_performance_results.csv', index=False)

## Regression on all data, one benchmark at a time

In [28]:
# Fit a regression to log_price ~ date + bench
bench = 'MMLU'
aa_df['price'] = aa_df['USD per 1M Tokens']
aa_df['log_price'] = np.log10(aa_df['price'])
aa_df['date'] = aa_df['Release Date'].map(lambda x: pd.Timestamp(x).toordinal())
aa_df['bench'] = aa_df[bench]

# Fit the model
model = smf.ols('log_price ~ date + bench', data=aa_df).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:              log_price   R-squared:                       0.523
Model:                            OLS   Adj. R-squared:                  0.506
Method:                 Least Squares   F-statistic:                     30.72
Date:                Thu, 13 Feb 2025   Prob (F-statistic):           9.86e-10
Time:                        18:33:02   Log-Likelihood:                -50.122
No. Observations:                  59   AIC:                             106.2
Df Residuals:                      56   BIC:                             112.5
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept   2213.9100    291.378      7.598      0.0

In [29]:
print(model.params)
print(model.conf_int())


Intercept    2213.910022
date           -0.002999
bench           0.032741
dtype: float64
                     0            1
Intercept  1630.209488  2797.610557
date         -0.003789    -0.002208
bench         0.021287     0.044194


In [30]:
# Date parameter
date_param = model.params['date'] * 365  # Convert daily to annual
annual_factor = int(round(10**(-date_param)))  # Convert log slope to factor
print(f'It costs {annual_factor}x less each year to keep {bench} performance fixed.')

It costs 12x less each year to keep MMLU performance fixed.


In [31]:
model.conf_int().loc['date'][0]


-0.0037894678198683793

In [39]:
results = []
for bench in benchmarks + ['LMSys Chatbot Arena ELO']:
    print(f'{bench}')
    df = aa_df.copy()
    df = df.dropna(subset=[bench])
    df['bench'] = df[bench]

    model = smf.ols('log_price ~ date + bench', data=df).fit()
    # Print number of observations and R-squared
    print(f'Number of observations: {len(df)}')
    print(f'R-squared: {model.rsquared:.2f}')
    date_param = model.params['date'] * 365  # Convert daily to annual
    date_param_ci = model.conf_int(alpha=0.1).loc['date']
    date_param_ci_low = date_param_ci[0] * 365
    date_param_ci_high = date_param_ci[1] * 365
    annual_factor = int(round(10**(-date_param)))  # Convert log slope to factor
    annual_factor_ci_high = int(round(10**(-date_param_ci_low)))  # Convert log slope to factor
    annual_factor_ci_low = int(round(10**(-date_param_ci_high)))  # Convert log slope to factor

    min_date = pd.Timestamp.fromordinal(df["date"].min()).date()
    max_date = pd.Timestamp.fromordinal(df["date"].max()).date()
    date_range = max_date - min_date
    results.append({
        'bench': bench,
        'sample_size': len(df),
        'start_date': min_date,
        'end_date': max_date,
        'price_reduction_factor_per_year_mean': annual_factor,
        'price_reduction_factor_per_year_ci': [annual_factor_ci_low, annual_factor_ci_high],
        'r_squared': round(model.rsquared, 2),
    })

    print(f'The price to achieve a fixed level of {bench} performance fell at a rate of')
    print(f'{annual_factor}x [{annual_factor_ci_low}, {annual_factor_ci_high}] per year')
    print(f'over {date_range.days / 365:.1f} years ({min_date} to {max_date})')
    print()


results_subdir = results_dir + 'aa_all_data_single_benchmark/'
os.makedirs(results_subdir, exist_ok=True)
single_benchmark_regression_results_df = pd.DataFrame(results)
single_benchmark_regression_results_df.to_csv(results_subdir + 'single_benchmark_regression_results.csv', index=False)

MMLU
Number of observations: 59
R-squared: 0.52
The price to achieve a fixed level of MMLU performance fell at a rate of
12x [7, 22] per year
over 3.2 years (2021-11-20 to 2025-02-01)

GPQA Diamond
Number of observations: 52
R-squared: 0.49
The price to achieve a fixed level of GPQA Diamond performance fell at a rate of
13x [6, 31] per year
over 1.9 years (2023-03-01 to 2025-02-01)

MATH 5
Number of observations: 44
R-squared: 0.23
The price to achieve a fixed level of MATH 5 performance fell at a rate of
11x [3, 40] per year
over 1.9 years (2023-03-01 to 2025-02-01)

MATH-500
Number of observations: 38
R-squared: 0.47
The price to achieve a fixed level of MATH-500 performance fell at a rate of
43x [13, 138] per year
over 1.9 years (2023-03-01 to 2025-02-01)

HumanEval
Number of observations: 38
R-squared: 0.51
The price to achieve a fixed level of HumanEval performance fell at a rate of
12x [5, 28] per year
over 1.9 years (2023-03-01 to 2025-02-01)

LMSys Chatbot Arena ELO
Number of o

In [40]:
bench = 'LMSys Chatbot Arena ELO'
df = aa_df.copy()
df = df.dropna(subset=[bench])
df['bench'] = df[bench]

model = smf.ols('log_price ~ date + bench', data=df).fit()
date_param = model.params['date'] * 365  # Convert daily to annual
date_param_ci = model.conf_int(alpha=0.1).loc['date']
date_param_ci_low = date_param_ci[0] * 365
date_param_ci_high = date_param_ci[1] * 365
annual_factor = int(round(10**(-date_param)))  # Convert log slope to factor
annual_factor_ci_high = int(round(10**(-date_param_ci_low)))  # Convert log slope to factor
annual_factor_ci_low = int(round(10**(-date_param_ci_high)))  # Convert log slope to factor

# Create a contour plot to visualize the regression model surface

# Create a grid of date and price values
date_range = np.linspace(
    df['date'].min(),
    df['date'].max(),
    100
)
price_range = np.linspace(
    df['log_price'].min(),
    df['log_price'].max(),
    100
)
date_mesh, price_mesh = np.meshgrid(date_range, price_range)

# Create prediction data
pred_data = pd.DataFrame({
    'date': date_mesh.ravel(),
    'log_price': price_mesh.ravel()
})

# Get predicted bench values
# Need to rearrange the regression equation to solve for bench
# log_price = b0 + b1*date + b2*bench
# bench = (log_price - b0 - b1*date) / b2
b0 = model.params['Intercept']
b1 = model.params['date']
b2 = model.params['bench']
pred_bench = (pred_data['log_price'] - b0 - b1*pred_data['date']) / b2

# Reshape predictions for contour plot
Z = pred_bench.to_numpy().reshape(date_mesh.shape)

# Create contour plot
fig = go.Figure(data=[
    go.Contour(
        x=date_range,
        y=10**price_range, # Convert back to regular price scale
        z=Z,
        contours=dict(
            # start=30,
            # end=90,
            # size=10,
            showlabels=False,
            labelfont=dict(size=12),
            coloring='lines'
        ),
        line=dict(width=2),
        colorbar=dict(
            title=bench,
            titleside='right',
        ),
        colorscale='Viridis',
        # zmin=30,  # Set consistent range
        # zmax=90,
        showscale=False,
    )
])

# Add actual data points
fig.add_trace(go.Scatter(
    x=df['date'],
    y=df['price'],
    text=df['Model Name'],
    mode='markers',
    name='Actual Data',
    marker=dict(
        color=df[bench],  # Color based on benchmark score
        colorscale='Viridis',
        colorbar=dict(
            title=f'{bench} score',
            titleside='right'
        ),
        size=8,
        showscale=True,
        # cmin=30,  # Set consistent range
        # cmax=90
    )
))

# Update layout
fig.update_layout(
    title=f'For fixed {bench}, the usage price of LLMs has fallen by {annual_factor}x per year',
    xaxis_title='Date',
    yaxis_title='Price (USD per 1M tokens)',
    width=800,
    height=600,
    margin=dict(l=50, r=50, t=50, b=50)  # Reduce margins
)

# Remove gridlines
fig.update_layout(
    xaxis=dict(
        # showgrid=False,
        tickfont=dict(size=12)  # Larger x-axis tick labels
    ),
    yaxis=dict(
        # showgrid=False,
        tickfont=dict(size=12)  # Larger y-axis tick labels
    )
)

# Convert x-axis ticks to dates
fig.update_xaxes(
    ticktext=[pd.Timestamp.fromordinal(int(d)).strftime('%Y-%m') for d in date_range[::20]],
    tickvals=date_range[::20]
)

# Use log scale for y-axis
fig.update_layout(yaxis_type='log')

if save:
    save_plot(fig, results_subdir, f'{bench}_contours', extensions=['png'])

fig.show()

## Regression on all data, all benchmarks at once

In [34]:
df = aa_df.copy()
# Rename benchmark columns to all lowercase, underscore separated
new_bench_cols = [col.lower().replace(' ', '_').replace('-', '_') for col in benchmarks]
df.rename(columns={bench: new_bench_cols[i] for i, bench in enumerate(benchmarks)}, inplace=True)
df = df.dropna(subset=new_bench_cols)

model = smf.ols('log_price ~ date + ' + ' + '.join(new_bench_cols), data=df).fit()
# Print number of observations and R-squared
print(f'Number of observations: {len(df)}')
print(f'R-squared: {model.rsquared:.2f}')
date_param = model.params['date'] * 365  # Convert daily to annual
date_param_ci = model.conf_int(alpha=0.1).loc['date']
date_param_ci_low = date_param_ci[0] * 365
date_param_ci_high = date_param_ci[1] * 365
annual_factor = int(round(10**(-date_param)))  # Convert log slope to factor
annual_factor_ci_high = int(round(10**(-date_param_ci_low)))  # Convert log slope to factor
annual_factor_ci_low = int(round(10**(-date_param_ci_high)))  # Convert log slope to factor

min_date = pd.Timestamp.fromordinal(df["date"].min()).date()
max_date = pd.Timestamp.fromordinal(df["date"].max()).date()
date_range = max_date - min_date
results = [{
    'benchmarks': benchmarks,
    'sample_size': len(df),
    'start_date': min_date,
    'end_date': max_date,
    'price_reduction_factor_per_year_mean': annual_factor,
    'price_reduction_factor_per_year_ci': [annual_factor_ci_low, annual_factor_ci_high],
    'r_squared': round(model.rsquared, 2),
}]

print(f'The price to achieve a fixed level of performance fell at a rate of')
print(f'{annual_factor}x [{annual_factor_ci_low}, {annual_factor_ci_high}] per year')
print(f'over {date_range.days / 365:.1f} years ({min_date} to {max_date})')
print()

results_subdir = results_dir + 'all_data_all_benchmarks/'
os.makedirs(results_subdir, exist_ok=True)
all_benchmarks_regression_results_df = pd.DataFrame(results)
all_benchmarks_regression_results_df.to_csv(results_subdir + 'all_benchmarks_regression_results.csv', index=False)

Number of observations: 31
R-squared: 0.79
The price to achieve a fixed level of performance fell at a rate of
16x [6, 47] per year
over 1.9 years (2023-03-01 to 2025-02-01)



## Regression on all data, on the average of benchmark scores

In [35]:
df = aa_df.copy()
# Rename benchmark columns to all lowercase, underscore separated
new_bench_cols = [col.lower().replace(' ', '_').replace('-', '_') for col in benchmarks]
df.rename(columns={bench: new_bench_cols[i] for i, bench in enumerate(benchmarks)}, inplace=True)
df = df.dropna(subset=new_bench_cols)
df['avg_bench'] = df[new_bench_cols].mean(axis=1)
df[['Model Name', 'Release Date', 'avg_bench']].sort_values(by='avg_bench', ascending=False).head(10)

,Model Name,Release Date,avg_bench
70,Gemini 2.0 Flash,2025-02-01,83.0
48,Gemini-1.5-Pro-2024-09,2024-09-01,78.0
60,Claude-3.5-Sonnet-2024-10,2024-10-01,74.4
65,Phi 4,2025-01-01,74.2
39,GPT-4o-2024-08,2024-08-01,72.0
28,GPT-4o-2024-05,2024-05-01,71.6
32,Claude-3.5-Sonnet-2024-06,2024-06-01,70.2
61,GPT-4o-2024-11,2024-11-01,69.8
49,Gemini-1.5-Flash-2024-09,2024-09-01,68.8
38,GPT-4o-mini,2024-07-01,68.0


In [36]:
model = smf.ols('log_price ~ date + avg_bench', data=df).fit()
# Print number of observations and R-squared
print(f'Number of observations: {len(df)}')
print(f'R-squared: {model.rsquared:.2f}')
date_param = model.params['date'] * 365  # Convert daily to annual
date_param_ci = model.conf_int(alpha=0.1).loc['date']
date_param_ci_low = date_param_ci[0] * 365
date_param_ci_high = date_param_ci[1] * 365
annual_factor = int(round(10**(-date_param)))  # Convert log slope to factor
annual_factor_ci_high = int(round(10**(-date_param_ci_low)))  # Convert log slope to factor
annual_factor_ci_low = int(round(10**(-date_param_ci_high)))  # Convert log slope to factor

min_date = pd.Timestamp.fromordinal(df["date"].min()).date()
max_date = pd.Timestamp.fromordinal(df["date"].max()).date()
date_range = max_date - min_date
results = [{
    'benchmarks': benchmarks,
    'sample_size': len(df),
    'start_date': min_date,
    'end_date': max_date,
    'price_reduction_factor_per_year_mean': annual_factor,
    'price_reduction_factor_per_year_ci': [annual_factor_ci_low, annual_factor_ci_high],
    'r_squared': round(model.rsquared, 2),
}]

print(f'The price to achieve a fixed level of performance fell at a rate of')
print(f'{annual_factor}x [{annual_factor_ci_low}, {annual_factor_ci_high}] per year')
print(f'over {date_range.days / 365:.1f} years ({min_date} to {max_date})')
print()

results_subdir = results_dir + 'aa_all_data_avg_benchmark/'
os.makedirs(results_subdir, exist_ok=True)
avg_benchmarks_regression_results_df = pd.DataFrame(results)
avg_benchmarks_regression_results_df.to_csv(results_subdir + 'avg_benchmarks_regression_results.csv', index=False)

Number of observations: 31
R-squared: 0.58
The price to achieve a fixed level of performance fell at a rate of
55x [18, 169] per year
over 1.9 years (2023-03-01 to 2025-02-01)



In [37]:
# Create a contour plot to visualize the regression model surface

# Create a grid of date and price values
date_range = np.linspace(
    df['date'].min(),
    df['date'].max(),
    100
)
price_range = np.linspace(
    df['log_price'].min(),
    df['log_price'].max(),
    100
)
date_mesh, price_mesh = np.meshgrid(date_range, price_range)

# Create prediction data
pred_data = pd.DataFrame({
    'date': date_mesh.ravel(),
    'log_price': price_mesh.ravel()
})

# Get predicted bench values
# Need to rearrange the regression equation to solve for bench
# log_price = b0 + b1*date + b2*bench
# bench = (log_price - b0 - b1*date) / b2
b0 = model.params['Intercept']
b1 = model.params['date']
b2 = model.params['avg_bench']
pred_bench = (pred_data['log_price'] - b0 - b1*pred_data['date']) / b2

# Reshape predictions for contour plot
Z = pred_bench.to_numpy().reshape(date_mesh.shape)

# Create contour plot
fig = go.Figure(data=[
    go.Contour(
        x=date_range,
        y=10**price_range,  # Convert back to regular price scale
        z=Z,
        contours=dict(
            start=30,
            end=90,
            size=10,
            showlabels=False,
            labelfont=dict(size=12),
            coloring='lines'
        ),
        line=dict(width=2),
        colorbar=dict(
            title='Average benchmark score',
            titleside='right',
        ),
        colorscale='Viridis',
        zmin=30,  # Set consistent range
        zmax=90,
        showscale=False,
    )
])

# Add actual data points
fig.add_trace(go.Scatter(
    x=df['date'],
    y=df['price'],
    text=df['Model Name'],
    mode='markers',
    name='Actual Data',
    marker=dict(
        color=df['avg_bench'],  # Color based on average benchmark score
        colorscale='Viridis',
        colorbar=dict(
            title='Average benchmark score',
            titleside='right'
        ),
        size=8,
        showscale=True,
        cmin=30,  # Set consistent range
        cmax=90
    )
))

# Update layout
fig.update_layout(
    title=f'For a fixed performance level, the usage price of LLMs has fallen by {annual_factor}x per year',
    xaxis_title='Date',
    yaxis_title='Price (USD per 1M tokens)',
    width=800,
    height=600,
    margin=dict(l=50, r=50, t=50, b=50)  # Reduce margins
)

# Remove gridlines
fig.update_layout(
    xaxis=dict(
        tickfont=dict(size=12)  # Larger x-axis tick labels
    ),
    yaxis=dict(
        tickfont=dict(size=12)  # Larger y-axis tick labels
    )
)

# Convert x-axis ticks to dates
fig.update_xaxes(
    ticktext=[pd.Timestamp.fromordinal(int(d)).strftime('%Y-%m') for d in date_range[::20]],
    tickvals=date_range[::20]
)

# Use log scale for y-axis
fig.update_layout(yaxis_type='log')

if save:
    save_plot(fig, results_subdir, 'avg_bench_contours', extensions=['png'])

fig.show()


## Regression on top models by price-performance on any benchmark, all benchmarks at once

## Summary and comparison of methods

In [38]:
# Compare the cheapest model results to the full regression results
for bench in benchmarks:
    print(f'{bench} trends')
    cheapest_model_result = cheapest_model_summary_df[cheapest_model_summary_df['bench'] == bench].iloc[0]
    top_n_price_performance_result = top_n_price_performance_results_df[top_n_price_performance_results_df['bench'] == bench].iloc[0]
    single_benchmark_regression_result = single_benchmark_regression_results_df[single_benchmark_regression_results_df['bench'] == bench].iloc[0]
    all_benchmarks_regression_result = all_benchmarks_regression_results_df.iloc[0]
    avg_benchmarks_regression_result = avg_benchmarks_regression_results_df.iloc[0]
    print(
        'Cheapest model at threshold:',
        cheapest_model_result['price_reduction_factor_per_year_geomean'],
        cheapest_model_result['price_reduction_factor_per_year_range'],
    )
    print(
        'Top model by price-performance:',
        top_n_price_performance_result['price_reduction_factor_per_year_mean'],
        top_n_price_performance_result['price_reduction_factor_per_year_ci'],
    )
    print(
        'Regression on this benchmark score:',
        single_benchmark_regression_result['price_reduction_factor_per_year_mean'],
        single_benchmark_regression_result['price_reduction_factor_per_year_ci'],
    )
    print(
        'Regression on all benchmark scores:',
        all_benchmarks_regression_result['price_reduction_factor_per_year_mean'],
        all_benchmarks_regression_result['price_reduction_factor_per_year_ci'],
    )
    print(
        'Regression on average benchmark score:',
        avg_benchmarks_regression_result['price_reduction_factor_per_year_mean'],
        avg_benchmarks_regression_result['price_reduction_factor_per_year_ci'],
    )
    print()

MMLU trends
Cheapest model at threshold: 21 [17, 28]
Top model by price-performance: 18 [11, 28]
Regression on this benchmark score: 12 [7, 22]
Regression on all benchmark scores: 16 [6, 47]
Regression on average benchmark score: 55 [18, 169]

GPQA Diamond trends
Cheapest model at threshold: 192 [87, 331]
Top model by price-performance: 88 [21, 367]
Regression on this benchmark score: 13 [6, 31]
Regression on all benchmark scores: 16 [6, 47]
Regression on average benchmark score: 55 [18, 169]

MATH 5 trends
Cheapest model at threshold: 41 [22, 76]
Top model by price-performance: 58 [26, 130]
Regression on this benchmark score: 11 [3, 40]
Regression on all benchmark scores: 16 [6, 47]
Regression on average benchmark score: 55 [18, 169]

MATH-500 trends
Cheapest model at threshold: 342 [22, 20513]
Top model by price-performance: 55 [15, 198]
Regression on this benchmark score: 43 [13, 138]
Regression on all benchmark scores: 16 [6, 47]
Regression on average benchmark score: 55 [18, 169]
